In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

import torch
import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Note Embedding

In [ ]:
# NEM
note_labels_num = 21
note_tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")
nem = BertForSequenceClassification.from_pretrained("bert-base-chinese", num_labels=note_labels_num).to(device)
state_dict = torch.load("models/NEM_v1.pth", map_location=device)
nem.load_state_dict(state_dict)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [ ]:
# Note embedding dictionary
df_ne = pd.read_csv("data/note_embedding.csv")
ne_list = df_ne.iloc[:, 1:].values.astype(float)

In [ ]:
def note2vec(note):
    # Convert note text to vector
    nem.eval()
    with torch.no_grad():
        encoding = note_tokenizer(
            note,
            truncation=True,
            padding='max_length', 
            max_length=64,
            return_tensors='pt'
        )
        input_ids = encoding['input_ids'].to(device)
        attention_mask = encoding['attention_mask'].to(device)
        outputs = nem.bert(input_ids=input_ids, attention_mask=attention_mask)
        embedding = outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()
    
    # Recommend similar note
    embedding = embedding.reshape(1, -1)
    similarities = cosine_similarity(embedding, ne_list)[0]
    idx = similarities.argmax()
    rc_vec = ne_list[idx]
    rc_name = df_ne.iloc[idx, 0]
    rc_similarity = similarities[idx]
    return rc_vec, rc_name, rc_similarity

In [6]:
def get_note_list_vec(note_list):
    vec_list = []
    for note in note_list:
        vec, name, similarity = note2vec(note)
        vec_list.append(vec)
        print(f"Note: {note} => {name} (similarity: {similarity:.4f})")
    return vec_list

# Perfume Vector

In [7]:
# Model
class PerfumeEmbedding(nn.Module):
    def __init__(self, note_dim=768, hidden=256, z_dim=128, num_classes=8, dropout=0.3):
        super().__init__()
        self.phi_top = nn.Sequential(
            nn.Linear(note_dim, hidden),
            nn.ReLU(),
            nn.LayerNorm(hidden),
            nn.Dropout(dropout)
        )
        self.phi_mid = nn.Sequential(
            nn.Linear(note_dim, hidden),
            nn.ReLU(),
            nn.LayerNorm(hidden),
            nn.Dropout(dropout)
        )
        self.phi_base = nn.Sequential(
            nn.Linear(note_dim, hidden),
            nn.ReLU(),
            nn.LayerNorm(hidden),
            nn.Dropout(dropout)
        )
        
        self.attn_top = nn.Sequential(
            nn.Linear(hidden, 64),
            nn.Tanh(),
            nn.Dropout(dropout),
            nn.Linear(64, 1)
        )
        self.attn_mid = nn.Sequential(
            nn.Linear(hidden, 64),
            nn.Tanh(),
            nn.Dropout(dropout),
            nn.Linear(64, 1)
        )
        self.attn_base = nn.Sequential(
            nn.Linear(hidden, 64),
            nn.Tanh(),
            nn.Dropout(dropout),
            nn.Linear(64, 1)
        )
        
        self.rho_top = nn.Sequential(
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Dropout(dropout)
        )
        self.rho_mid = nn.Sequential(
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Dropout(dropout)
        )
        self.rho_base = nn.Sequential(
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Dropout(dropout)
        )
        
        self.cross_attn = nn.MultiheadAttention(embed_dim=hidden, num_heads=4, batch_first=True, dropout=dropout)
        
        self.rho = nn.Sequential(
            nn.Linear(hidden * 6, z_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(z_dim, z_dim), 
            nn.ReLU()
        )
        self.classifier = nn.Linear(z_dim, num_classes)

    def aggregate(self, phi, attn_net, rho, notes):
        h = phi(notes)
        scores = attn_net(h).squeeze(-1)
        attn_weights = torch.softmax(scores, dim=1).unsqueeze(-1)
        weighted_h = (h * attn_weights).sum(dim=1)
        return rho(weighted_h)
    
    def forward(self, top_notes, mid_notes, base_notes):
        h_top = self.aggregate(self.phi_top, self.attn_top, self.rho_top, top_notes)
        h_mid = self.aggregate(self.phi_mid, self.attn_mid, self.rho_mid, mid_notes)
        h_base = self.aggregate(self.phi_base, self.attn_base, self.rho_base, base_notes)
        
        h_seq = torch.stack([h_top, h_mid, h_base], dim=1)
        h_interact, _ = self.cross_attn(h_seq, h_seq, h_seq)
        h_top_i, h_mid_i, h_base_i = h_interact.unbind(dim=1)
        
        h_all = torch.cat([h_top, h_mid, h_base, h_top_i, h_mid_i, h_base_i], dim=-1)
        
        z = self.rho(h_all)
        logits = self.classifier(z)
        return logits, z
    
pem = PerfumeEmbedding().to(device)
state_dict = torch.load("models/PEM_v2.pth")
pem.load_state_dict(state_dict)

<All keys matched successfully>

In [8]:
# Get perfume vector
def perfume2vec(top_notes, mid_notes, base_notes):
    pem.eval()
    with torch.no_grad():
        top_vecs = get_note_list_vec(top_notes)
        mid_vecs = get_note_list_vec(mid_notes)
        base_vecs = get_note_list_vec(base_notes)
                
        top_vecs = torch.tensor(np.array(top_vecs), dtype=torch.float32).to(device).unsqueeze(0)
        mid_vecs = torch.tensor(np.array(mid_vecs), dtype=torch.float32).to(device).unsqueeze(0)
        base_vecs = torch.tensor(np.array(base_vecs), dtype=torch.float32).to(device).unsqueeze(0)
        
        _, z = pem(top_vecs, mid_vecs, base_vecs)
    return z.cpu().numpy().squeeze()

# Recommend Perfumes

In [9]:
# Load perfume vector list
df_pe = pd.read_csv("data/perfume_embedding_nn.csv")
pe_list = df_pe.iloc[:, 1:].values.astype(float)

In [10]:
def get_recommended_perfumes(top_notes, mid_notes, base_notes, top_k=5):
    perfume_vec = perfume2vec(top_notes, mid_notes, base_notes).reshape(1, -1)
    similarities = cosine_similarity(perfume_vec, pe_list)[0]
    df_top = df_pe.assign(similarity=similarities).nlargest(top_k, 'similarity')
    recommendation = df_top[['name','similarity']].values.tolist()
    return recommendation

In [11]:
like_top_notes = ["橙子"]
like_mid_notes = ["咖啡"]
like_base_notes = ["巧克力"]
top_k = 10

rc_perfumes = get_recommended_perfumes(like_top_notes, like_mid_notes, like_base_notes, top_k)
display(rc_perfumes)

Note: 橙子 => 橙子 (similarity: 1.0000)
Note: 咖啡 => 咖啡 (similarity: 1.0000)
Note: 巧克力 => 巧克力 (similarity: 1.0000)


[['Mugler Angel Fantasm 黑天使幻想女性淡香精', 0.9306738356113713],
 ['Aquolina Pink Sugar Creamy Sunshine 奶油阳光女性淡香水', 0.9237883703457287],
 ['Ariana Grande Sweet Like Candy 女性淡香精', 0.9224516461840185],
 ['Juicy Couture Viva La Juicy Sucre 蛋糕甜心女性淡香精', 0.9219706607464045],
 ['Juicy Couture Viva La Juicy Sucre 蛋糕甜心女性淡香精 TESTER', 0.9219706607464045],
 ['Juicy Couture Viva La Juicy Sucre 蛋糕甜心女性淡香精行动香氛', 0.9219706607464045],
 ['Burberry Goddess 缪斯女神淡香精', 0.9205307121399737],
 ['Burberry Goddess 缪斯女神淡香精迷你瓶', 0.9096613683790785],
 ['Aquolina Pink Sugar Red Velvet 红丝绒女性淡香水', 0.9047305913803726],
 ['Fcuk Friction Her 爱火女性淡香精', 0.9036712784692651]]

# Perfume Detail

In [ ]:
# Show perfume details
df_perfume = pd.read_csv("data/1976_clean.csv")
rc_names = [item[0] for item in rc_perfumes]
df_rc_perfume = df_perfume[df_perfume['name'].isin(rc_names)]
df_rc_perfume = df_rc_perfume.set_index('name').loc[rc_names].reset_index()
display(df_rc_perfume)

,name,description,original_fragrance,top_notes,middle_notes,base_notes,detail_url,gender,fragrance
0,Mugler Angel Fantasm 黑天使幻想女性淡香精,这是一款多汁、令人垂涎且大胆的香水，可以释放我们最深的幻想。,诱人美食调,凤梨、佛手柑,椰子、栀子花,广藿香、香草、琥珀,https://www.1976.com.tw/prod/22882,female,美食调
1,Aquolina Pink Sugar Creamy Sunshine 奶油阳光女性淡香水,又轻又亮！柑橘和椰奶的香气融入含羞草、依兰和蒂亚雷的花香中心，留下香草、焦糖和莫诺伊的珍贵香气。,诱人美食调,椰奶、青柑、柑橘、佛手柑,方糖、蒂亚蕾花、依兰依兰、栀子花、含羞草,焦糖、香草、莫诺伊油、檀香、苔藓,https://www.1976.com.tw/prod/21813,female,美食调
2,Ariana Grande Sweet Like Candy 女性淡香精,对本蚂蚁而言，这瓶的味道实在是太棒了！甜甜的闻著觉得整个人都软软的（？）喜欢甜味的人可以收！,花果甜美调,黑莓、义大利佛手柑、水梨,鲜奶油、茉莉、金银花、鸡蛋花,黑醋栗、棉花糖、香草、松果,https://www.1976.com.tw/prod/14870,female,果香调
3,Juicy Couture Viva La Juicy Sucre 蛋糕甜心女性淡香精,千金大小姐风格的可爱香氛！一开始是强烈的栀子花和黑醋栗香，接著可以慢慢闻到柑橘和甜而不腻的奶...,诱人美食调,栀子花、柑橘、红醋栗,橙花、茉莉花、桃子、杏仁奶油,奶油、香草、檀香、可可,https://www.1976.com.tw/prod/14903,female,美食调
4,Juicy Couture Viva La Juicy Sucre 蛋糕甜心女性淡香精 TE...,"超爱甜味，更爱奶味！这款简直就是我的理想~加上华丽的外观,仿佛是一个被宠坏了的小公主,超级可...",诱人美食调,栀子花、柑橘、红醋栗,橙花、茉莉花、桃子、杏仁奶油,奶油、香草、檀香、可可,https://www.1976.com.tw/prod/15492,female,美食调
5,Juicy Couture Viva La Juicy Sucre 蛋糕甜心女性淡香精行动香氛,这支真的是非常甜的一支香水 喜欢甜香的应该会很爱～甜甜的美食调 闻起来感觉很幸福 并且香水的...,诱人美食调,栀子花、柑橘、红醋栗,橙花、茉莉花、桃子、杏仁奶油,奶油、香草、檀香、可可,https://www.1976.com.tw/prod/21305,female,美食调
6,Burberry Goddess 缪斯女神淡香精,当初冲著是Cara跟Kate Moss代言的，喷上去会有一种进入不同时区的感觉，一种知性、温...,诱人美食调,香草、薰衣草、可可、姜,香草鱼子酱,纯香草,https://www.1976.com.tw/prod/21155,female,美食调
7,Burberry Goddess 缪斯女神淡香精迷你瓶,当初冲著是Cara跟Kate Moss代言的，喷上去会有一种进入不同时区的感觉，一种知性、温...,诱人美食调,香草、薰衣草、可可和姜,香草鱼子酱,纯香草,https://www.1976.com.tw/prod/21410,female,美食调
8,Aquolina Pink Sugar Red Velvet 红丝绒女性淡香水,AQUOLINA在2021年推出第一款香水－Pink Sugar Red Velvet 红丝...,诱人美食调,草莓、佛手柑,糖果、紫罗兰、橙花、百合花,鲜奶油、香草、琥珀,https://www.1976.com.tw/prod/21573,female,美食调
9,Fcuk Friction Her 爱火女性淡香精,成熟版的奶油椰子乖乖，一整个就是融化的奶油一样诱人，花香不明显，睡前喷一些，留香度可以到隔天...,诱人美食调,金银花、苹果花、红浆果,香草、奶油椰子、粉红茉莉花、棉花糖,麝香、白香根草、檀香木、琥珀、金发伍兹,https://www.1976.com.tw/prod/9194,female,美食调
